# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing Turi Create functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

In [1]:
import numpy as np
import pandas as pd
import math
import string

In [68]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

## Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [3]:
products = pd.read_csv('amazon_baby.csv')

In [4]:
products.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


### Build the word count vector for each review
Convert each review to lower-case, and remove the punctuation.

In [5]:
products.iloc[269]

name      The First Years Massaging Action Teether
review                    A favorite in our house!
rating                                           5
Name: 269, dtype: object

Now, we will perform 2 simple data transformations:

1. Remove punctuation using regular expression
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

1. Remove punctuation using regular expression

**IMPORTANT**. Make sure to fill n/a values in the **review** column with empty strings (if applicable). The n/a values indicate empty reviews. For instance, Pandas's the fillna() method lets you replace all N/A's in the **review** columns as follows:

In [6]:
products[products['review'].isnull()]

,name,review,rating
38,SoftPlay Twinkle Twinkle Elmo A Bedtime Book,NaN,5
58,Our Baby Girl Memory Book,NaN,5
721,"Summer Infant, Ultimate Training Pad - Twin Ma...",NaN,5
1050,Safety 1st Deluxe 4-in-1 Bath Station,NaN,1
1183,Northstate Superyard Playgate Light Gray,NaN,5
1373,Munchkin Mozart Magic Cube,NaN,5
1688,"Graco TotBloc Pack 'N Play with Carry Bag, Bug...",NaN,5
1689,"Graco TotBloc Pack 'N Play with Carry Bag, Bug...",NaN,5
2372,Prince Lionheart Table Edge Guard with 4 Corne...,NaN,2
2409,BABYBJORN Toilet Trainer - White/Red,NaN,5


In [7]:
products['review'].isnull().sum()

829

In [8]:
products = products.fillna({'review':''}) # fill in N/A's in the review column

In [9]:
products['review'].isnull().sum()

0

In [10]:
import re

# Remove punctuation characters
# match everything that is not a lowercase a-z, uppercase A-Z, or digits 0-9, and replaces them with a space.
def remove_punctuation(text):
    text = text.lower()
    return re.sub(r"[^a-zA-Z0-9]", " ", str(text))

products['review_clean'] = products['review'].apply(remove_punctuation)

## methods 2
# def remove_punctuation(text):
#     import string
#     return text.translate(None, string.punctuation) 

# products['review_clean'] = products['review'].apply(remove_punctuation)

In [11]:
products['review_clean'].head()

0    these flannel wipes are ok  but in my opinion ...
1    it came early and was not disappointed  i love...
2    very soft and comfortable and warmer than it l...
3    this is a product well worth the purchase   i ...
4    all of my kids have cried non stop when i trie...
Name: review_clean, dtype: object

2. Transform the reviews into word-counts.

In [63]:
# def word_count(sentence):
#     w_list = sentence.lower().split()
#     w_count = dict()
    
#     for w in w_list:
#         w_count[w] = w_count.get(w, 0) + 1.0
        
#     return w_count

In [64]:
# products['word_count'] = review_without_punctuation.apply(word_count)

In [65]:
# products.head()

,name,review,rating,word_count,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,"{'it': 3.0, 'came': 1.0, 'early': 1.0, 'and': ...",1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,"{'very': 1.0, 'soft': 1.0, 'and': 2.0, 'comfor...",1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,"{'this': 4.0, 'is': 4.0, 'a': 2.0, 'product': ...",1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,"{'all': 2.0, 'of': 1.0, 'my': 1.0, 'kids': 2.0...",1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,"{'when': 2.0, 'the': 6.0, 'binky': 3.0, 'fairy...",1


Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the word_count column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [9]:
# products.iloc[269]['word_count']

{'a': 1.0, 'favorite': 1.0, 'in': 1.0, 'our': 1.0, 'house': 1.0}

### Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [12]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [13]:
products['sentiment'] = products['rating'].apply(lambda rating: +1 if rating > 3 else -1)

products.head(10)

,name,review,rating,review_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,this is a product well worth the purchase i ...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,all of my kids have cried non stop when i trie...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,when the binky fairy came to our house we did...,1
6,A Tale of Baby's Days with Peter Rabbit,"Lovely book, it's bound tightly so you may not...",4,lovely book it s bound tightly so you may not...,1
7,"Baby Tracker&reg; - Daily Childcare Journal, S...",Perfect for new parents. We were able to keep ...,5,perfect for new parents we were able to keep ...,1
8,"Baby Tracker&reg; - Daily Childcare Journal, S...",A friend of mine pinned this product on Pinter...,5,a friend of mine pinned this product on pinter...,1
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4,this has been an easy way for my nanny to reco...,1
10,"Baby Tracker&reg; - Daily Childcare Journal, S...",I love this journal and our nanny uses it ever...,4,i love this journal and our nanny uses it ever...,1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

### Split data into training and test sets
Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [14]:
products.columns

Index(['name', 'review', 'rating', 'review_clean', 'sentiment'], dtype='object')

In [15]:
y = products['sentiment']
X = products.drop(['sentiment'], axis=1)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, stratify=y, random_state=1)

print(len(X_train))
print(len(X_test))

133401
33351


In [23]:
!ls

CLA02-NB01.ipynb                   amazon_baby.sframe
W1_Sentiment-Predicting.ipynb      module-2-assignment-test-idx.json
amazon_baby.csv                    module-2-assignment-train-idx.json
amazon_baby.gl


In [32]:
import json

with open('module-2-assignment-train-idx.json') as json_file:
    train_idx = json.load(json_file)
    
with open('module-2-assignment-test-idx.json') as json_file:
    test_idx = json.load(json_file)   

In [42]:
train_idx[:20]

[0, 1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 15, 16, 17, 19, 20, 21, 22, 23]

In [41]:
train_data = products.iloc[train_idx]
test_data = products.iloc[test_idx]

In [55]:
print(len(train_data), len(test_data))

133416 33336


### Build the word count vector for each review  
We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as **bag-of-word features**. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

* Learn a vocabulary (set of all words) from the training data. Only the words that show up in the training data will be considered for feature extraction.

* Compute the occurrences of the words in each review and collect them into a row vector.

* Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix **train_matrix**.

* Using the same mapping between words and columns, convert the test data into a sparse matrix **test_matrix**.

The following cell uses `CountVectorizer` in scikit-learn. Notice the **token_pattern** argument in the constructor.

In [44]:
from sklearn.feature_extraction.text import CountVectorizer

In [45]:
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
# Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

In [46]:
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='\\b\\w+\\b', tokenizer=None,
                vocabulary=None)

In [47]:
train_matrix, test_matrix

(<133416x57445 sparse matrix of type '<class 'numpy.int64'>'
 	with 7431855 stored elements in Compressed Sparse Row format>,
 <33336x57445 sparse matrix of type '<class 'numpy.int64'>'
 	with 1856335 stored elements in Compressed Sparse Row format>)

In [76]:
train_array = train_matrix.toarray()

In [48]:
print(test_matrix)

  (0, 1881)	1
  (0, 2558)	1
  (0, 3878)	1
  (0, 4295)	1
  (0, 4519)	1
  (0, 5405)	1
  (0, 5998)	2
  (0, 6920)	1
  (0, 7002)	2
  (0, 13307)	1
  (0, 17651)	1
  (0, 18987)	1
  (0, 21169)	1
  (0, 24017)	1
  (0, 24153)	1
  (0, 24214)	1
  (0, 24707)	1
  (0, 24983)	2
  (0, 25568)	3
  (0, 26014)	1
  (0, 26441)	1
  (0, 27290)	2
  (0, 27315)	1
  (0, 27368)	1
  (0, 27900)	1
  :	:
  (33335, 19504)	1
  (33335, 21679)	1
  (33335, 24214)	1
  (33335, 25568)	2
  (33335, 27368)	1
  (33335, 28179)	1
  (33335, 30286)	1
  (33335, 31011)	1
  (33335, 31412)	1
  (33335, 33125)	1
  (33335, 33789)	1
  (33335, 34789)	1
  (33335, 39376)	1
  (33335, 39652)	1
  (33335, 40847)	1
  (33335, 44723)	1
  (33335, 49012)	1
  (33335, 50208)	1
  (33335, 50504)	2
  (33335, 50800)	1
  (33335, 50854)	1
  (33335, 50968)	1
  (33335, 54678)	1
  (33335, 55735)	1
  (33335, 56661)	1


Keep in mind that the test data must be transformed in the same way as the training data.

## Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. We should create an instance of the [LogisticRegression class](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) and then call the method `fit()` to train the classifier. This model should use the sparse word count matrix (**train_matrix**)as features and the column **sentiment** of train_data as the target. Use the default values for other parameters. Call this model `sentiment_model`.

In [57]:
sentiment_model = LinearRegression()
sentiment_model.fit(train_matrix, train_data['sentiment'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [58]:
print(len(sentiment_model.coef_))
sentiment_model.coef_

57445


array([-0.07624343, -0.12644704,  0.16097678, ...,  0.0173788 ,
       -0.18990994,  0.93812441])

In [71]:
sentiment_model.intercept_

0.6282252612088051

There should be over 100,000 coefficients in this sentiment_model. Recall from the lecture that positive weights w_j correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. Calculate the number of positive (>= 0, which is actually nonnegative) coefficients. 

In [62]:
len([i for i in sentiment_model.coef_ if i >= 0])

30517